In [ ]:
# TODO 0: import
import pandas as pd
from geopy.distance import geodesic

In [ ]:
def calculate_distance(row):
    """
    计算两个经纬度坐标之间的距离

    :param coord1: 第一个坐标，格式为 (纬度, 经度)
    :param coord2: 第二个坐标，格式为 (纬度, 经度)
    :return: 距离（单位：千米）
    """
    coord1 = row['distance1']
    coord2 = row['distance2']

    if coord1 ==('nan',) or coord2 == ('nan',):
        distance = ''
    else:
        distance = geodesic(coord1, coord2).kilometers
    return distance


def split_to_tuple(row):
    """
    定义一个函数，将逗号分隔的字符串拆分成元组
    """
    row = str(row).split(',')
    row.reverse()
    return tuple(row)



In [ ]:
# TODO 1: read file
read_branch = 'abfss://test-data@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/test/WZX/03-workshop/branch_address.xlsx'
read_workshop = 'abfss://test-data@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/test/WZX/03-workshop/workshop_address.xlsx'
df_branch = pd.read_excel(read_branch)
df_workshop = pd.read_excel(read_workshop,dtype=str)

# TODO 2：obtain_longitude_and_latitude
df_branch['distance1'] = df_branch['latitude_and_longitude'].apply(split_to_tuple)
df_workshop['distance2'] = df_workshop['latitude_and_longitude'].apply(split_to_tuple)
df_branch = df_branch[['branch_name','distance1']]
df_workshop = df_workshop[['client_name','distance2']]
print(df_workshop)

# TODO 3: calculate_distance
df_merge = pd.merge(df_branch.assign(key=1),df_workshop.assign(key=1),on='key').drop('key',axis=1)
print(df_merge.columns)
df_merge['distance'] = df_merge[['distance1','distance2']].apply(calculate_distance,axis=1)
df_result = df_merge[['client_name','branch_name','distance']]
print(df_result)

# display(df_branch['distance'].tolist())



In [ ]:
# 导包
from notebookutils import mssparkutils  
from openpyxl import load_workbook
import pandas as pd
import os
from datetime import datetime,timedelta    

# 挂载blob
mssparkutils.fs.mount(
    "abfss://test-data@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn",
    "/mnt/test",
    {"LinkedService" : "AzureBlobStorage_proddataplatcn3blob01"}
)

path = mssparkutils.fs.getMountPath("/mnt/test")

# 获得处理的文件
root = path + '/test/WZX/03-workshop/address_distance.xlsx'
# files = os.listdir(root)
# file_path = os.path.join(root,files[0])
print(root)

In [ ]:
print(df_result)

In [ ]:
# TODO 4：save
df_result.to_csv(root,header=True,index=False)